In [29]:
import os
import re

import numpy as np
import pandas as pd

In [52]:
current_file = "data/fz10_2024_12.xlsx"  #<-------------------------------------------------

xls = pd.ExcelFile(current_file)
file_name = os.path.splitext(os.path.basename(current_file))[0]
parts = current_file.split("_")
date_str = f"{parts[-2]}-{parts[-1].split('.')[0]}"

df = pd.read_excel(
    current_file,
    sheet_name=xls.sheet_names[0],
    header=0,
    skiprows=6,
    usecols=None
)
xls.sheet_names
df.head(10)


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44
0,NaN,NaN,NaN,Insgesamt,NaN,NaN,mit Dieselantrieb,NaN,NaN,mit Hybridantrieb \n(incl. Plug-in-Hybrid),...,NaN,mit Elektroantrieb (BEV),NaN,NaN,mit Allradantrieb,NaN,NaN,Cabriolets,NaN,NaN
1,NaN,Marke,Modellreihe,Dezember 2024,Jan.-Dezember 2024,Anteil \nin %,Dezember 2024,Jan.-Dezember 2024,Anteil \nin %,Dezember 2024,...,Anteil \nin %,Dezember 2024,Jan.-Dezember 2024,Anteil \nin %,Dezember 2024,Jan.-Dezember 2024,Anteil \nin %,Dezember 2024,Jan.-Dezember 2024,Anteil \nin %
2,NaN,AIWAYS,U5,-,25,92.6,-,-,-,-,...,-,-,25,92.6,-,-,-,-,-,-
3,NaN,NaN,SONSTIGE,-,2,7.4,-,-,-,-,...,-,-,/,/,-,-,-,-,-,-
4,NaN,AIWAYS ZUSAMMEN,NaN,-,27,0,-,-,-,-,...,-,-,27,0,-,-,-,-,-,-
5,NaN,ALFA ROMEO,GIULIA,81,1462,24,11,111,8.1,-,...,-,-,-,-,54,1151,32.3,-,-,-
6,NaN,NaN,JUNIOR,114,378,6.2,-,-,-,106,...,-,8,130,100,-,-,-,-,-,-
7,NaN,NaN,STELVIO,127,2184,35.8,39,818,59.7,-,...,-,-,-,-,127,2183,61.2,-,-,-
8,NaN,NaN,TONALE,129,2074,34,19,442,32.2,110,...,-,-,-,-,18,233,6.5,-,-,-
9,NaN,ALFA ROMEO ZUSAMMEN,NaN,451,6098,0.2,69,1371,0.3,216,...,-,8,130,0,199,3567,0.5,-,-,-


## Start DATA cleaning
Remove all columns with "Jan. - Oktober 2025" pattern

In [53]:
df_clean = df.dropna(axis=1, how='all')
df_clean = df.iloc[:, 1:]
######################################################
pattern_to_drop_col = df.at[1, 'Unnamed: 4']
patterns = [pattern_to_drop_col, "Anteil \nin %"]
######################################################
patterns_norm = [re.sub(r"\s+", " ", p) for p in patterns]
first_row = df_clean.iloc[1, :].astype(str).str.replace(r"\s+", " ", regex=True)
keep = ~first_row.isin(patterns_norm)

df_clean = df_clean.loc[:, keep]
cols = df_clean.iloc[0, :].astype(str).str.replace(r"\s+", " ", regex=True)
df_clean.columns = cols

df_clean = df_clean.iloc[1:].reset_index(drop=True)
#################################################
df_clean["date"] = f"{date_str}-01"
#################################################
df_clean = df_clean.iloc[1:].reset_index(drop=True)
df_clean["date"] = pd.to_datetime(df_clean["date"])

cols = list(df_clean.columns)
cols[0], cols[1] = 'Marke', 'Modellreihe'
df_clean.columns = cols
df_clean['Marke'] = df_clean['Marke'].ffill()

df_clean = df_clean[~df_clean["Marke"].astype(str).str.contains("ZUSAMMEN", case=False, na=False)]
df_clean = df_clean.reset_index(drop=True)

df_clean = df_clean.rename(columns={
    "Marke": "brand",
    "Modellreihe": "model",
    "Insgesamt": "total",
    "mit Dieselantrieb": "diesel_engine",
    "mit Hybridantrieb (incl. Plug-in-Hybrid)": "hybrid_incl_plugin",
    "Benzin-Hybridantrieb (incl. Plug-in-Hybrid)": "petrol_hybrid_incl_plugin",
    "Diesel-Hybridantrieb (incl. Plug-in-Hybrid)": "diesel_hybrid_incl_plugin",
    "Hybridantrieb (ohne Plug-in-Hybrid)": "hybrid_excl_plugin",
    "Benzin-Hybridantrieb (ohne Plug-in-Hybrid)": "petrol_hybrid_excl_plugin",
    "Diesel-Hybridantrieb (ohne Plug-in-Hybrid)": "diesel_hybrid_excl_plugin",
    "Plug-in-Hybridantrieb": "plugin_hybrid",
    "Benzin-Plug-in-Hybridantrieb": "petrol_plugin_hybrid",
    "Diesel-Plug-in-Hybridantrieb": "diesel_plugin_hybrid",
    "mit Elektroantrieb (BEV)": "electric_bev",
    "mit Allradantrieb": "all_wheel_drive",
    "Cabriolets": "convertibles",
    "date": "date"
})

# Bottom trim
df_clean = df_clean.iloc[:-4, :].reset_index(drop=True)

# Define the desired order
cols = ['brand', 'model', 'date'] + [c for c in df_clean.columns if c not in ['brand', 'model', 'date']]

# Reorder
df_clean = df_clean[cols]

# Replace all placeholders for missing data with NaN
pd.set_option('future.no_silent_downcasting', True)

df_clean.iloc[:, 3:] = (
    df_clean.iloc[:, 3:]
    .replace(['-', '–', '—', ''], np.nan)
)

# Convert all numeric columns to Int64
for col in df_clean.columns[3:]:
    df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce').astype('Int64')

# Check


df_clean.head(10)


,brand,model,date,total,diesel_engine,hybrid_incl_plugin,petrol_hybrid_incl_plugin,diesel_hybrid_incl_plugin,hybrid_excl_plugin,petrol_hybrid_excl_plugin,diesel_hybrid_excl_plugin,plugin_hybrid,petrol_plugin_hybrid,diesel_plugin_hybrid,electric_bev,all_wheel_drive,convertibles
0,AIWAYS,U5,2024-12-01,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,AIWAYS,SONSTIGE,2024-12-01,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,ALFA ROMEO,GIULIA,2024-12-01,81,11,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,54,<NA>
3,ALFA ROMEO,JUNIOR,2024-12-01,114,<NA>,106,106,<NA>,106,106,<NA>,<NA>,<NA>,<NA>,8,<NA>,<NA>
4,ALFA ROMEO,STELVIO,2024-12-01,127,39,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,127,<NA>
5,ALFA ROMEO,TONALE,2024-12-01,129,19,110,110,<NA>,92,92,<NA>,18,18,<NA>,<NA>,18,<NA>
6,ALPINE,A110,2024-12-01,33,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
7,ALPINE,SONSTIGE,2024-12-01,12,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
8,ASTON MARTIN,DBX,2024-12-01,4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,4,<NA>
9,ASTON MARTIN,V8,2024-12-01,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [54]:
df_clean.tail(10)

,brand,model,date,total,diesel_engine,hybrid_incl_plugin,petrol_hybrid_incl_plugin,diesel_hybrid_incl_plugin,hybrid_excl_plugin,petrol_hybrid_excl_plugin,diesel_hybrid_excl_plugin,plugin_hybrid,petrol_plugin_hybrid,diesel_plugin_hybrid,electric_bev,all_wheel_drive,convertibles
406,VW,TIGUAN,2024-12-01,5260,2377,2291,2291,<NA>,1562,1562,<NA>,729,729,<NA>,<NA>,1708,<NA>
407,VW,TOUAREG,2024-12-01,450,381,41,41,<NA>,<NA>,<NA>,<NA>,41,41,<NA>,<NA>,450,<NA>
408,VW,TOURAN,2024-12-01,1085,627,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
409,VW,TRANSPORTER,2024-12-01,2918,2522,46,46,<NA>,<NA>,<NA>,<NA>,46,46,<NA>,<NA>,264,<NA>
410,VW,UP,2024-12-01,17,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,14,<NA>,<NA>
411,VW,SONSTIGE,2024-12-01,26,<NA>,15,15,<NA>,13,13,<NA>,2,2,<NA>,<NA>,9,<NA>
412,XPENG,G6,2024-12-01,53,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,53,34,<NA>
413,XPENG,G9,2024-12-01,41,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,41,38,<NA>
414,XPENG,P7,2024-12-01,5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,5,3,<NA>
415,XPENG,SONSTIGE,2024-12-01,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [55]:
# Save the cleaned dataframe to the data folder
output_filename = f"{file_name}_cleaned.xlsx"
output_path = f"data/{output_filename}"

df_clean.to_excel(output_path, index=False, sheet_name="Cleaned Data")

print(f"✓ Data successfully saved to: {output_path}")
print(f"✓ Total rows: {len(df_clean)}")
print(f"✓ Total columns: {len(df_clean.columns)}")

✓ Data successfully saved to: data/fz10_2024_12_cleaned.xlsx
✓ Total rows: 416
✓ Total columns: 17
